In [1]:
!git clone https://github.com/geohot/tinygrad.git

fatal: destination path 'tinygrad' already exists and is not an empty directory.


In [2]:
cd tinygrad/

/home/ahmad-mujtaba/Ahmad_Mujtaba/python_work/Tinygrad_Learning_Projects/tinygrad


In [3]:
!python3 -m pip install -e .

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Obtaining file:///home/ahmad-mujtaba/Ahmad_Mujtaba/python_work/Tinygrad_Learning_Projects/tinygrad
  Attempting uninstall: tinygrad
    Found existing installation: tinygrad 0.6.0
    Uninstalling tinygrad-0.6.0:
      Successfully uninstalled tinygrad-0.6.0
  Running setup.py develop for tinygrad


In [3]:
pwd

'/home/ahmad-mujtaba/Ahmad_Mujtaba/python_work/Tinygrad_Learning_Projects'

In [2]:
import time, sys
import json
import random
from PIL import Image
import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import optim, BatchNorm2d
import tinygrad.nn as nn
from tinygrad.helpers import flatten
from tinygrad.nn.optim import SGD, Adam
from sklearn.datasets import fetch_openml
from tinygrad.state import safe_save, safe_load, get_state_dict, load_state_dict, torch_load

In [9]:
cd ../

/home/ahmad-mujtaba/Ahmad_Mujtaba/python_work/Tinygrad_Learning_Projects


In [4]:
dataset_dir = 'Dataset'

images = []
labels = []

# Iterate through the cat images
cat_dir = os.path.join(dataset_dir, 'Cat')
for filename in os.listdir(cat_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(cat_dir, filename)
        image = Image.open(image_path).convert('L')  
        image = image.resize((150, 150))
        images.append(np.array(image))
        labels.append(0)

In [5]:
dog_dir = os.path.join(dataset_dir, 'Dog')
for filename in os.listdir(dog_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(dog_dir, filename)
        image = Image.open(image_path).convert('L')
        image = image.resize((150, 150))
        images.append(np.array(image))
        labels.append(1)

/home/ahmad-mujtaba/Ahmad_Mujtaba/python_work/Tinygrad_Learning_Projects/venv/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:866: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [6]:
data = list(zip(images, labels))
random.shuffle(data)

shuffled_images, shuffled_labels = zip(*data)

shuffled_images = np.array(shuffled_images)
shuffled_labels = np.array(shuffled_labels)

shuffled_images = Tensor(shuffled_images)
shuffled_labels = Tensor(shuffled_labels)

In [7]:
shuffled_images.shape, shuffled_labels.shape

((24998, 150, 150), (24998,))

In [8]:
class CustomTinyNet:
    def __init__(self):
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=512, kernel_size=3)
        self.fc1 = nn.Linear(in_features=512*15*15, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=1)

    def forward(self, x):
        x = x.reshape((x.shape[0], 1, 150, 150))
        x = self.conv1(x)
        x = x.relu()
        x = x.max_pool2d(2)  
        x = x.dropout(0.2)  

        x = self.conv2(x)
        x = x.relu()
        x = x.max_pool2d(2)  
        x = x.dropout(0.2) 

        x = self.conv3(x)
        x = x.relu()
        x = x.max_pool2d(2)  
        x = x.dropout(0.5) 

        x = self.conv4(x)
        x = x.relu()
        x = x.dropout(0.2) 

        x = x.reshape(x.shape[0], -1)

        x = self.fc1(x) 
        x = x.relu()

        x = self.fc2(x) 
        return x.sigmoid()

net = CustomTinyNet()

In [9]:
Tensor.training = True

In [10]:
def binary_cross_entropy(out, Y):
    epsilon = 1e-7 
    loss = -Y * Tensor.log(out + epsilon) - (1 - Y) * Tensor.log(1 - out + epsilon)
    loss = [float(l.numpy()[0]) for l in loss]
    
    return Tensor(np.mean(loss))


In [12]:
opt = SGD([net.conv1.weight, net.conv2.weight, net.conv3.weight, net.conv4.weight], lr=0.01)

In [13]:
num_epochs = 1000

running_loss, correct, total = 0.0, 0.0, 0.0
start_time = time.time()

file_write = open("output.txt", "a")

for epoch in range(num_epochs):
    # Randomly sample a batch
    samp = np.random.randint(0, shuffled_images.shape[0], size=(64))
    batch = Tensor(shuffled_images.numpy()[samp].astype('float32') / 255.0, requires_grad=False)
    # Get the corresponding labels
    labels_batch = shuffled_labels.numpy()[samp]

    # Forward pass
    out = net.forward(batch)

    # Compute loss
    loss = binary_cross_entropy(out, labels_batch)
    
    # Zero gradients
    opt.zero_grad()

    # Backward pass
    loss.backward()

    # Calculate accuracy
    pred = np.argmax(out.numpy().data, axis=-1)
    acc = (pred == labels_batch).mean()

    if epoch % 100 == 0:
        print(f"Time Taken: {time.time()-start_time:.1f}s, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.numpy():.5f}, Accuracy: {acc:.5f}")
        
        file_write.write(f"Epochs: {epoch+1}/{num_epochs}\tLoss: {loss.numpy():.5f}\tAccuracy: {acc:.5f}\tActual-label: {labels_batch}\tPrediction: {pred}\n")
        start_time = time.time()
    

file_write.close()


Time Taken: 3.2s, Epoch [1/1000], Loss: 0.69500, Accuracy: 0.45312
Time Taken: 97.0s, Epoch [101/1000], Loss: 0.68808, Accuracy: 0.62500
Time Taken: 97.2s, Epoch [201/1000], Loss: 0.69268, Accuracy: 0.51562
Time Taken: 96.0s, Epoch [301/1000], Loss: 0.69764, Accuracy: 0.35938
Time Taken: 96.4s, Epoch [401/1000], Loss: 0.69222, Accuracy: 0.53125
Time Taken: 97.1s, Epoch [501/1000], Loss: 0.69476, Accuracy: 0.46875
Time Taken: 97.0s, Epoch [601/1000], Loss: 0.69561, Accuracy: 0.45312
Time Taken: 97.4s, Epoch [701/1000], Loss: 0.69395, Accuracy: 0.43750
Time Taken: 97.1s, Epoch [801/1000], Loss: 0.69386, Accuracy: 0.48438
Time Taken: 97.5s, Epoch [901/1000], Loss: 0.69178, Accuracy: 0.54688


In [15]:
# set training flag to false
Tensor.training = False

st = time.perf_counter()
avg_acc = 0
for step in range(1000):
  # random sample a batch
  samp = np.random.randint(0, shuffled_images.shape[0], size=(64))
  batch = Tensor(shuffled_images.numpy()[samp].astype('float32') / 255.0, requires_grad=False)
  
  # get the corresponding labels
  batch_labels = shuffled_labels.numpy()[samp]

  # forward pass
  out = net.forward(batch)

  # calculate accuracy
  pred = np.argmax(out.numpy(), axis=-1)

  labels = [label for label in batch_labels]
  avg_acc += (pred == labels).mean()

print(f"Test Accuracy: {avg_acc / 1000}")
print(f"Time Taken To Test: {time.perf_counter() - st}")

Test Accuracy: 0.498703125
Time Taken To Test: 896.732031386


In [16]:
# first we need the state dict of our model
state_dict = get_state_dict(net)

# then we can just save it to a file
safe_save(state_dict, "model/model.safetensors")

In [20]:
import tinygrad.tensor as tensor
from PIL import Image
# Step 1: Preprocess the image
image = Image.open('Dataset/Dog/0.jpg').convert('L')
image = image.resize((150, 150))

# Step 2: Load the trained model
state_dict = safe_load("model/model.safetensors")

checkpoints = {
    'conv1.weight': state_dict['conv1.weight'],
    'conv1.bias': state_dict['conv1.bias'],
    'conv2.weight': state_dict['conv2.weight'],
    'conv2.bias': state_dict['conv2.bias'],
    'conv3.weight': state_dict['conv3.weight'],
    'conv3.bias': state_dict['conv3.bias'],
    'conv4.weight': state_dict['conv4.weight'],
    'conv4.bias': state_dict['conv4.bias'],
    'fc1.weight': state_dict['fc1.weight'],
    'fc1.bias': state_dict['fc1.bias'],
    'fc2.weight': state_dict['fc2.weight'],
    'fc2.bias': state_dict['fc2.bias']
}

model = CustomTinyNet()

load_state_dict(model, checkpoints)


image = np.array(image)

image = image.astype(np.float32) / 255.0  

start_time = time.time()
preprocessed_image = image.reshape((1, 1, 150, 150))

# Step 3: Forward pass
input_tensor = tensor.Tensor(preprocessed_image)  
output_tensor = net.forward(input_tensor)  

print(f"\n\nTime Taken to Predict the class: {time.time() - start_time: .4f}")

# Step 4: Obtain predictions
predicted_class= np.argmax(output_tensor.numpy(), axis=-1)
print("Predicted class:", predicted_class)

ram used:  1.14 GB, fc2.bias                                          : 100%|██████████| 12/12 [00:00<00:00, 294.65it/s]


loaded weights in 43.86 ms, 1.14 GB loaded at 25.88 GB/s


Time Taken to Predict the class:  0.0181
Predicted class: [0]
